In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel("timetable.xlsx")

In [3]:
df.head()

,Unnamed: 0,Monday,Tuesday,Wednesday,Thursday,Friday
0,8:30 - 10:30,BCT-Lab-A-DDA-406A,BCT-Lab-B-DDA-406B / CSS-Lab-II-AVN-410B,AIML-Lab-A-AK-608 / DA-Lab-A-VK-607 / CC-Lab-A...,HMI-Lab-C-VW-410 / CE-Lab-E-MP-503 / CE-Lab-F-...,Major Project
1,10:30 - 12:30,BCT-Theory-DDA-405 / AIH-Lab-N-L11-603,AIH-Theory-SK-703 / BDA-Lab-C-AH-404 / ADV-Lab...,HMI-Lab-B-DRK-412 / AIML-Theory-AK-407 / DBMS-...,HMI-Theory-VW-405 / DA-Lab-B-VK-703 / CE-Lab-C...,DL-Lab-M-L11-410
2,12:30 - 1:30,Break,Break,Break,Break,Break
3,1:30 - 3:30,ADV-Theory-L6-703 / AIH-Theory-SK-405 / ADV-La...,NLP-Theory-PJB-405 / BDA-Theory-AH-407 / CSS-A...,ADV-Theory-L6-703 / CA-Theory-DDA-405 / BDA-La...,BDA-Lab-D-AH-404 / CSS-Lab-VIII-AVN-410 / ADV-...,SMA-Theory-PJB-601 / DL-Theory-L11-405 / RS-Th...
4,3:30 - 5:30,ADV-Lab-A-L6-801AppleLab / AIH-Lab-D-SK-406B /...,ADV-Lab-B-L6-801AppleLab / ADV-Lab-G-L5-408 / ...,ADV-Lab-C-L6-801 / ADV-Lab-H-L5-410 / AIH-Lab-...,ADV-Lab-D-L6-801 / AIH-Lab-G-SK-406 / SMA-Lab-...,ADV-Lab-E-L5-406 / AIH-Lab-H-SK-406 / SMA-Lab-...


In [10]:
import pandas as pd

def parse_time_slot(time_str):
    """Split a time range string '8:30 - 10:30' into start_time and end_time."""
    start_time, end_time = time_str.split(" - ")
    return start_time.strip(), end_time.strip()

def parse_entry(entry):
    """Split entries separated by '/' and extract subject, session type, batch (if available), faculty initials, and room number."""
    if entry == "Break" or pd.isnull(entry):
        return []
    
    # Split multiple sessions if separated by '/'
    sessions = entry.split('/')
    parsed_sessions = []
    
    for session in sessions:
        parts = session.strip().split('-')
        subject = parts[0] if len(parts) > 0 else None
        session_type = parts[1] if len(parts) > 1 else None
        batch = parts[2] if len(parts) == 5 else None  # Only parse batch if there are 5 segments
        faculty = parts[-2] if len(parts) > 3 else None
        room = parts[-1] if len(parts) > 3 else None
        
        parsed_sessions.append((subject, session_type, batch, faculty, room))
    
    return parsed_sessions

# Initialize list to store the transformed rows
transformed_data = []

# Mapping for days (Monday to Friday)
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']

# Dictionary to store previous entry for potential merging
previous_entry = {}

# Iterate over each row in the original CSV
for idx, row in df.iterrows():
    # Parse the time slot
    start_time, end_time = parse_time_slot(row['Unnamed: 0'])
    
    # Loop over each day column
    for day in days:
        entry = row[day]
        parsed_sessions = parse_entry(entry)
        
        # Process each session in the current time slot
        for subject, session_type, batch, faculty, room in parsed_sessions:
            # Skip if it's a break or invalid session
            if subject is None:
                continue
            
            # Construct the entry key for checking consecutive times
            entry_key = (day, subject, session_type, batch, faculty, room)
            
            # Check if the previous entry matches the current one for merging
            if entry_key in previous_entry:
                # Update the end time of the existing entry
                previous_entry[entry_key]['end_time'] = end_time
            else:
                # Create a new entry and add it to the transformed data
                new_entry = {
                    'timetable_id': len(transformed_data) + 1,
                    'start_time': start_time,
                    'end_time': end_time,
                    'day': day,
                    'subject_id': subject,
                    'session_type': session_type,
                    'lecture_batch_id': batch if session_type == "Theory" else None,
                    'lab_batch_id': batch if session_type == "Lab" else None,
                    'room_number': room,
                    'faculty_id': faculty
                }
                transformed_data.append(new_entry)
                previous_entry[entry_key] = new_entry  # Store for potential merging

# Convert the list of rows into a DataFrame
transformed_df = pd.DataFrame(transformed_data)

# Save the transformed data to a new CSV file
output_file = 'transformed_schedule.csv'
transformed_df.to_csv(output_file, index=False)

print(f"Transformed data saved to {output_file}")


Transformed data saved to transformed_schedule.csv


In [11]:
transformed_df

,timetable_id,start_time,end_time,day,subject_id,session_type,lecture_batch_id,lab_batch_id,room_number,faculty_id
0,1,8:30,10:30,Monday,BCT,Lab,None,A,406A,DDA
1,2,8:30,10:30,Tuesday,BCT,Lab,None,B,406B,DDA
2,3,8:30,10:30,Tuesday,CSS,Lab,None,II,410B,AVN
3,4,8:30,10:30,Wednesday,AIML,Lab,None,A,608,AK
4,5,8:30,10:30,Wednesday,DA,Lab,None,A,607,VK
...,...,...,...,...,...,...,...,...,...,...
88,89,3:30,5:30,Friday,ADV,Lab,None,E,406,L5
89,90,3:30,5:30,Friday,AIH,Lab,None,H,406,SK
90,91,3:30,5:30,Friday,SMA,Lab,None,P,408,PJB
91,92,3:30,5:30,Friday,CSS,Lab,None,V,604,AVN
